In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re 
import numpy as np

# *** trash

In [3]:
trash = pd.read_csv('../data/trash_hauler_report.csv')

In [4]:
trash = trash.rename(columns = {'Request ':'request','Request Number':'request_number','Date Opened':'date_opened','Description':'description','Incident Address':'incident_address','Zip Code':'zipcode','Trash Hauler':'trash_hauler','Trash Route':'trash_route','Council District':'council_district','State Plan X':'state_plan_x','State Plan Y': 'state_plan_y'}) 


In [5]:
# data cleaning.

trash['trash_hauler'] = trash['trash_hauler'].str.title()
trash['incident_address'] = trash['incident_address'].str.title()
trash['description'] = trash['description'].str.lower() 

In [6]:
trash.trash_hauler.value_counts().to_frame()

,count
trash_hauler,
Red River,14395
Metro,3580
Waste Ind,1350


# *** missed_pu

In [8]:
missed_pu = trash.loc[trash['request'].str.contains("Missed Pickup")]

In [9]:
missed_pu.head()

,request_number,date_opened,request,description,incident_address,zipcode,trash_hauler,trash_route,council_district,state_plan_x,state_plan_y
1,25274,11/01/17,Trash - Curbside/Alley Missed Pickup,curb/trash miss tuesday.,4028 Clarksville Pike,37218.0,Red River,4202,1.0,1.721259e+06,685444.799565
2,25276,11/01/17,Trash - Curbside/Alley Missed Pickup,curb/trash miss tuesday.,6528 Thunderbird Dr,37209.0,Red River,4205,20.0,1.707027e+06,659887.471571
3,25307,11/01/17,Trash - Curbside/Alley Missed Pickup,missed,2603 Old Matthews Rd,37207.0,Waste Ind,2206,2.0,1.735692e+06,685027.245923
4,25312,11/01/17,Trash - Curbside/Alley Missed Pickup,missed the even side of the road.,604 Croley Dr,37209.0,Red River,4203,20.0,1.710186e+06,664205.101066
8,25330,11/01/17,Trash - Curbside/Alley Missed Pickup,missed.,4484 Lavergne Couchville Pike,37013.0,Red River,4210,33.0,1.794534e+06,618749.342732


In [10]:
trash.request.value_counts().to_frame().reset_index()

,request,count
0,Trash - Curbside/Alley Missed Pickup,15028
1,Trash - Backdoor,2629
2,Trash Collection Complaint,2312
3,Damage to Property,257


# *** trash_complaints

In [12]:
trash_complaints = trash.loc[trash['request'] == "Trash Collection Complaint"]

# *** missed_pu_2

In [14]:
missed_pu_2= trash.loc[~trash['request'].str.contains('Missed Pickup')]

In [15]:
missed_pu_2.head()

,request_number,date_opened,request,description,incident_address,zipcode,trash_hauler,trash_route,council_district,state_plan_x,state_plan_y
0,25270,11/01/17,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,Red River,3205,2.0,1.727970e+06,686779.478089
5,25317,11/01/17,Trash Collection Complaint,left trash cart in middle of driveway instead ...,3602 Floral Dr,37211.0,Red River,4304,16.0,1.751660e+06,643215.201073
6,25325,11/01/17,Damage to Property,trash/emptied wednesday & now metal black-mail...,4721 Chalmers Dr,37215.0,Red River,3303,34.0,1.726301e+06,637078.402245
7,25327,11/01/17,Trash Collection Complaint,"trash out on time, miss again tuesday. alley",1816 Jo Johnston Ave,37203.0,Metro,9208,21.0,1.731459e+06,666013.601229
11,25350,11/01/17,Trash Collection Complaint,missed- 4th week in a row.,3210 Whites Creek Pike,37207.0,Red River,3201,3.0,1.726433e+06,691919.200296


In [16]:
missed_pu_2 = missed_pu_2.dropna(subset = 'description') 

In [17]:
missed_pu_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5196 entries, 0 to 20218
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   request_number    5196 non-null   int64  
 1   date_opened       5196 non-null   object 
 2   request           5196 non-null   object 
 3   description       5196 non-null   object 
 4   incident_address  5195 non-null   object 
 5   zipcode           5177 non-null   float64
 6   trash_hauler      4902 non-null   object 
 7   trash_route       4873 non-null   object 
 8   council_district  5184 non-null   float64
 9   state_plan_x      5189 non-null   float64
 10  state_plan_y      5189 non-null   float64
dtypes: float64(4), int64(1), object(6)
memory usage: 487.1+ KB


In [18]:
missed_pu_2 = missed_pu_2.loc[missed_pu_2['description'].str.contains('miss|missed|left')]

In [19]:
missed_pu.request_number.nunique()

15028

In [20]:
if missed_pu['request_number'].equals(missed_pu_2['request_number']):
        print("The two DataFrames are equal") 
else:
    print("The two DataFrames are not equal")

The two DataFrames are not equal


In [21]:
all_missed_pickups = pd.concat([missed_pu,missed_pu_2])  

In [22]:
all_missed_pickups.value_counts('description').to_frame()

,count
description,
miss,907
missed,633
missed.,622
cart still out,289
1 cart,229
...,...
it looks as if other neighbors had their trash picked up but somehow ours was missed today. we appreciate the good service from the trash collectors.,1
"it looks like a bag was taken off the top, but the rest was left in the can. i need it emptied asap.",1
it looks like all of lombardy ave was missed for trash pickup,1


# *** red_river

In [24]:
red_river = all_missed_pickups.loc[all_missed_pickups['trash_hauler'] == 'Red River']

In [25]:
red_river['zipcode'].dropna()

1        37218.0
2        37209.0
4        37209.0
8        37013.0
10       37218.0
          ...   
20167    37218.0
20169    37218.0
20178    37204.0
20179    37013.0
20202    37214.0
Name: zipcode, Length: 12990, dtype: float64

# *** red_river2

In [27]:
red_river2 = red_river.incident_address.value_counts().to_frame().reset_index()

In [28]:
red_river2

,incident_address,count
0,"12546 Old Hickory Blvd, Nashville, Tn 37013, U...",19
1,3710 N Natchez Ct,18
2,"6007 Obrien Ave, Nashville, Tn 37209, United S...",18
3,5135 Hickory Hollow Pkwy,18
4,"802 Crescent Rd, Nashville, Tn 37205, United S...",17
...,...,...
9168,"745 Newberry Rd, Nashville, Tn 37205, United S...",1
9169,708 Milbridge Ct,1
9170,"411 Woodmont Hall Pl, Nashville, Tn 37205, Uni...",1
9171,"2801 Valley Rd, Nashville, Tn 37215, United St...",1


# How much was each trash hauler fined for missed pickups?

In [30]:
all_missed_pickups_by_trash_hauler = all_missed_pickups['trash_hauler'].value_counts()

In [31]:
all_missed_pickups_by_trash_hauler.info

<bound method Series.info of trash_hauler
Red River    13018
Metro         3151
Waste Ind     1174
Name: count, dtype: int64>

In [32]:
all_missed_pickups_by_trash_hauler = pd.DataFrame(all_missed_pickups_by_trash_hauler)

In [33]:
all_missed_pickups_by_trash_hauler

,count
trash_hauler,
Red River,13018
Metro,3151
Waste Ind,1174


In [34]:
all_missed_pickups_by_trash_hauler['fines_total'] = all_missed_pickups_by_trash_hauler['count'] * 200

In [35]:
# ANSWER:

all_missed_pickups_by_trash_hauler

,count,fines_total
trash_hauler,,
Red River,13018,2603600
Metro,3151,630200
Waste Ind,1174,234800


# What routes had the most missed pickups?

In [37]:
all_missed_pickups['trash_route'].value_counts().head()

trash_route
4504    333
3302    292
4404    258
9303    249
4304    244
Name: count, dtype: int64

In [38]:
amp_by_route = all_missed_pickups['trash_route'].value_counts().head(10)

In [39]:
amp_by_route = pd.DataFrame(amp_by_route)

# ANSWER

In [41]:
amp_by_route.head()

,count
trash_route,
4504,333
3302,292
4404,258
9303,249
4304,244


# What are the different complaint types and which one had the most?

In [43]:
complaint_types_str = all_missed_pickups.value_counts('request')

In [44]:
complaint_types_str

request
Trash - Curbside/Alley Missed Pickup    15028
Trash - Backdoor                         2029
Trash Collection Complaint               1035
Damage to Property                         19
Name: count, dtype: int64

In [45]:
complaint_types = pd.DataFrame(complaint_types_str)

# ANSWER

In [47]:
complaint_types

,count
request,
Trash - Curbside/Alley Missed Pickup,15028
Trash - Backdoor,2029
Trash Collection Complaint,1035
Damage to Property,19


In [48]:
# Downloaded all_missed_pickups

In [49]:
complaint_types.to_csv('complaint_types.csv')

In [50]:
all_missed_pickups['zipcode'] = all_missed_pickups['zipcode'].fillna(0)

In [51]:
all_missed_pickups['zipcode'] = all_missed_pickups['zipcode'].astype(int)

# What is causing route 4504 to have the most missed pickups?

# *** all_missed_route_4504

In [54]:
all_missed_route_4504 = all_missed_pickups.loc[all_missed_pickups['trash_route'] == '4504']

In [55]:
# statistical info on value counts by address.

all_missed_route_4504['incident_address'].value_counts().describe()

count    209.000000
mean       1.593301
std        1.522950
min        1.000000
25%        1.000000
50%        1.000000
75%        2.000000
max       18.000000
Name: count, dtype: float64

In [56]:
all_missed_pickups['incident_address'].value_counts().describe()

count    12608.000000
mean         1.435755
std          1.120770
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max         21.000000
Name: count, dtype: float64

# *** all_missed_pickups_top_10_by_address

In [58]:
all_missed_pickups_top_10_by_address = all_missed_pickups['incident_address'].value_counts().head(10)

In [59]:
all_missed_pickups_top_10_by_address

incident_address
5135 Hickory Hollow Pkwy                                      21
6007 Obrien Ave, Nashville, Tn 37209, United States           19
3710 N Natchez Ct                                             19
12546 Old Hickory Blvd, Nashville, Tn 37013, United States    19
802 Crescent Rd, Nashville, Tn 37205, United States           18
1816 Jo Johnston Ave, Nashville, Tn 37203, United States      16
110 George L Davis Blvd                                       15
607 Estes Rd, Nashville, Tn 37215, United States              15
617 Kinsey Blvd                                               14
320 Old Hickory Blvd, Nashville, Tn 37221, United States      13
Name: count, dtype: int64

# All missed pickups by company and top address missed.

# *** all_missed_red_river

In [62]:
all_missed_red_river = all_missed_pickups.loc[all_missed_pickups['trash_hauler'] == 'Red River']

In [63]:
all_missed_red_river.head(3)

,request_number,date_opened,request,description,incident_address,zipcode,trash_hauler,trash_route,council_district,state_plan_x,state_plan_y
1,25274,11/01/17,Trash - Curbside/Alley Missed Pickup,curb/trash miss tuesday.,4028 Clarksville Pike,37218,Red River,4202,1.0,1.721259e+06,685444.799565
2,25276,11/01/17,Trash - Curbside/Alley Missed Pickup,curb/trash miss tuesday.,6528 Thunderbird Dr,37209,Red River,4205,20.0,1.707027e+06,659887.471571
4,25312,11/01/17,Trash - Curbside/Alley Missed Pickup,missed the even side of the road.,604 Croley Dr,37209,Red River,4203,20.0,1.710186e+06,664205.101066


# *** all_missed_red_river_address

In [65]:
all_missed_red_river_address = all_missed_red_river['incident_address'].value_counts()

In [66]:
all_missed_red_river_address = pd.DataFrame(all_missed_red_river_address)

In [67]:
all_missed_red_river_address.head()

,count
incident_address,
"12546 Old Hickory Blvd, Nashville, Tn 37013, United States",19
3710 N Natchez Ct,18
"6007 Obrien Ave, Nashville, Tn 37209, United States",18
5135 Hickory Hollow Pkwy,18
"802 Crescent Rd, Nashville, Tn 37205, United States",17


# *** all_missed_waste_ind

In [69]:
all_missed_waste_ind = all_missed_pickups.loc[all_missed_pickups['trash_hauler'] == 'Waste Ind']

In [70]:
all_missed_waste_ind = pd.DataFrame(all_missed_waste_ind)

In [71]:
all_missed_waste_ind.head(3)

,request_number,date_opened,request,description,incident_address,zipcode,trash_hauler,trash_route,council_district,state_plan_x,state_plan_y
3,25307,11/01/17,Trash - Curbside/Alley Missed Pickup,missed,2603 Old Matthews Rd,37207,Waste Ind,2206,2.0,1.735692e+06,685027.245923
94,25972,11/03/17,Trash - Curbside/Alley Missed Pickup,missed.,916 Petway Ave,37206,Waste Ind,2509,5.0,1.746483e+06,675576.311639
192,27229,11/09/17,Trash - Curbside/Alley Missed Pickup,trash missed past 2 weeks,415 35Th Ave N,37209,Waste Ind,3306,21.0,1.724396e+06,663795.769164


# *** all_missed_waste_ind_address

In [73]:
all_missed_waste_ind_address = all_missed_waste_ind['incident_address'].value_counts()

In [74]:
all_missed_waste_ind_address = pd.DataFrame(all_missed_waste_ind_address)

In [75]:
all_missed_waste_ind_address.head()

,count
incident_address,
15 Hermitage Ave,12
518 Monroe St,8
123 37Th Ave N,8
"613 Troy Ct, Nashville, Tn 37207, United States",8
1011 Elm Hill Pike,7


# *** all_missed_metro

In [77]:
all_missed_metro = all_missed_pickups.loc[all_missed_pickups['trash_hauler'] == 'Metro']

In [78]:
all_missed_metro = pd.DataFrame(all_missed_metro)

In [79]:
all_missed_metro.head(3)

,request_number,date_opened,request,description,incident_address,zipcode,trash_hauler,trash_route,council_district,state_plan_x,state_plan_y
9,25331,11/01/17,Trash - Curbside/Alley Missed Pickup,"trash missed tuesday again, alley",1206 Ireland St,37208,Metro,9208,19.0,1.733029e+06,668762.640891
12,25359,11/01/17,Trash - Curbside/Alley Missed Pickup,missed. does not have access to alley.,830 Meridian St,37207,Metro,9207,5.0,1.741745e+06,675328.666057
22,25471,11/01/17,Trash - Curbside/Alley Missed Pickup,"alley trash miss tuesday, also others as well,...",1904 Meharry Blvd,37208,Metro,9208,21.0,1.730048e+06,668927.190485


# *** all_missed_metro_address

In [81]:
all_missed_metro_address = all_missed_metro['incident_address'].value_counts()

In [82]:
all_missed_metro_address = pd.DataFrame(all_missed_metro_address)

In [83]:
all_missed_metro_address.head()

,count
incident_address,
"1816 Jo Johnston Ave, Nashville, Tn 37203, United States",16
2813 Delaware Ave,11
2801 Delaware Ave,11
110 George L Davis Blvd,11
1402 Lillian St,11


# What is the biggest factor causing trash to be missed for red river's route with the most missed pickups?

In [85]:
# How many addresses only had 1 miss?

all_missed_red_river_address.value_counts(all_missed_red_river_address['count'] == 1)

count
True     7039
False    2134
Name: count, dtype: int64

In [86]:
# Which routes has the most misses?

all_missed_red_river['trash_route'].value_counts()

trash_route
4504     333
3302     291
4404     258
4304     244
1303     241
        ... 
2405S      3
3303S      2
2505S      2
4201S      2
1502S      1
Name: count, Length: 125, dtype: int64

In [87]:
# Red river trash route value counts described.  There is something here.  
# The difference between the percentiles is much bigger than in previous analysis.

all_missed_red_river['trash_route'].value_counts().describe()

count    125.000000
mean     104.128000
std       76.931995
min        1.000000
25%       42.000000
50%       93.000000
75%      166.000000
max      333.000000
Name: count, dtype: float64

# *** rr_trash_route_4504

In [89]:
all_missed_red_river.loc[all_missed_red_river['trash_route'] == '4504'].head()

,request_number,date_opened,request,description,incident_address,zipcode,trash_hauler,trash_route,council_district,state_plan_x,state_plan_y
326,28863,11/17/17,Trash - Curbside/Alley Missed Pickup,they skipped my house for trash pickup,605 Welshcrest Pl,37013,Red River,4504,32.0,1.776990e+06,632123.549540
344,29146,11/20/17,Trash - Curbside/Alley Missed Pickup,missed. states has been missed twice in two we...,1312 Rice Hill Cir,37013,Red River,4504,32.0,1.779104e+06,631484.695500
398,30044,11/27/17,Trash - Curbside/Alley Missed Pickup,missed pick up - others missed in area,760 Pippin Dr,37013,Red River,4504,28.0,1.775550e+06,629604.762170
680,33636,12/15/17,Trash - Curbside/Alley Missed Pickup,missed.,4229 Kevinwood Ct,37013,Red River,4504,28.0,1.772752e+06,627876.811341
697,33779,12/15/17,Trash - Curbside/Alley Missed Pickup,missed pickup,5550 Mt View Rd,37013,Red River,4504,32.0,1.780680e+06,622557.601894


In [90]:
rr_trash_route_4504 = all_missed_red_river.loc[all_missed_red_river['trash_route'] == '4504']

In [91]:
rr_trash_route_4504.head()

,request_number,date_opened,request,description,incident_address,zipcode,trash_hauler,trash_route,council_district,state_plan_x,state_plan_y
326,28863,11/17/17,Trash - Curbside/Alley Missed Pickup,they skipped my house for trash pickup,605 Welshcrest Pl,37013,Red River,4504,32.0,1.776990e+06,632123.549540
344,29146,11/20/17,Trash - Curbside/Alley Missed Pickup,missed. states has been missed twice in two we...,1312 Rice Hill Cir,37013,Red River,4504,32.0,1.779104e+06,631484.695500
398,30044,11/27/17,Trash - Curbside/Alley Missed Pickup,missed pick up - others missed in area,760 Pippin Dr,37013,Red River,4504,28.0,1.775550e+06,629604.762170
680,33636,12/15/17,Trash - Curbside/Alley Missed Pickup,missed.,4229 Kevinwood Ct,37013,Red River,4504,28.0,1.772752e+06,627876.811341
697,33779,12/15/17,Trash - Curbside/Alley Missed Pickup,missed pickup,5550 Mt View Rd,37013,Red River,4504,32.0,1.780680e+06,622557.601894


# Why does 5135 hickory hollow pkwy have so many more missed pickups?

# *** amprrtr5404_by_address

In [94]:
amprrtr5404_by_address = rr_trash_route_4504['incident_address'].value_counts()

In [95]:
amprrtr5404_by_address

incident_address
5135 Hickory Hollow Pkwy                                    18
5550 Mt View Rd                                              7
4206 Moss Rd, Nashville, Tn 37013, United States             6
5159 Hickory Hollow Pkwy                                     6
1752 Bridgecrest Dr, Nashville, Tn 37013, United States      5
                                                            ..
2940 Baby Ruth Ln, 37013                                     1
1070 Rural Hill Rd, Nashville, Tn 37013, United States       1
1968 Wild Oaks Ct, Nashville, Tn 37013, United States        1
2816 Old Franklin Rd, Nashville, Tn 37013, United States     1
5062 Mt View Rd, Nashville, Tn 37013, United States          1
Name: count, Length: 209, dtype: int64

In [96]:
# Why is the max almost 10% of the total count?

rr_trash_route_4504['incident_address'].value_counts().describe()

count    209.000000
mean       1.593301
std        1.522950
min        1.000000
25%        1.000000
50%        1.000000
75%        2.000000
max       18.000000
Name: count, dtype: float64

# ASNWER:  This address is at the dead end of a street that is difficult to turn out of.

# $$$: amp_rr_tr_45_dead_end

In [99]:
amp_rr_tr_45_dead_end = rr_trash_route_4504.loc[rr_trash_route_4504['incident_address'] == '5135 Hickory Hollow Pkwy'].head(5)

In [100]:
amp_rr_tr_45_dead_end.head()

,request_number,date_opened,request,description,incident_address,zipcode,trash_hauler,trash_route,council_district,state_plan_x,state_plan_y
4822,83379,07/16/18,Trash - Curbside/Alley Missed Pickup,missed- caller is at a dead end,5135 Hickory Hollow Pkwy,37013,Red River,4504,32.0,1.771715e+06,625525.306114
5158,87010,07/31/18,Trash - Curbside/Alley Missed Pickup,missed- caller is at a dead end,5135 Hickory Hollow Pkwy,37013,Red River,4504,32.0,1.771716e+06,625524.026077
5756,93160,08/27/18,Trash - Curbside/Alley Missed Pickup,missed- caller is at a dead end,5135 Hickory Hollow Pkwy,37013,Red River,4504,32.0,1.771715e+06,625525.306114
5912,95192,09/05/18,Trash - Curbside/Alley Missed Pickup,missed- caller is at a dead end,5135 Hickory Hollow Pkwy,37013,Red River,4504,32.0,1.771715e+06,625525.306114
6461,104553,10/01/18,Trash - Curbside/Alley Missed Pickup,missed- trash again last friday caller is at a...,5135 Hickory Hollow Pkwy,37013,Red River,4504,32.0,1.771717e+06,625521.535039
